
# HIT140 Assessment 3 — Bat vs Rat: Full Colab Workflow
Setup → Load → Clean → EDA → Tests → Feature Engineering → Outputs

> Fill in the short interpretation prompts after each analysis block.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os, pathlib
BASE = "/content/drive/MyDrive/bat_rat_project"
for p in [
    f"{BASE}/notebooks",
    f"{BASE}/data",
    f"{BASE}/outputs/A/figs", f"{BASE}/outputs/A/tables",
    f"{BASE}/outputs/B/figs", f"{BASE}/outputs/B/tables",
]:
    pathlib.Path(p).mkdir(parents=True, exist_ok=True)


## 1) Setup and Configuration

In [3]:

# Paths. Update if your files are elsewhere (e.g., Drive).
DATASET1_PATH = "/content/dataset1.csv"
DATASET2_PATH = "/content/dataset2.csv"
OUTPUT_DIR = "/content/outputs"

import os
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Helper for saving single-plot figures
def save_current_plot(name):
    path = f"{OUTPUT_DIR}/{name}.png"
    plt.savefig(path, bbox_inches="tight")
    plt.close()
    print(f"Saved: {path}")


## 2) Load Data

In [4]:

d1 = pd.read_csv(DATASET1_PATH)
d2 = pd.read_csv(DATASET2_PATH)

print("dataset1 shape:", d1.shape)
print("dataset2 shape:", d2.shape)

display(d1.head())
display(d2.head())

print("\nDTypes 1:\n", d1.dtypes)
print("\nDTypes 2:\n", d2.dtypes)


dataset1 shape: (907, 12)
dataset2 shape: (2123, 7)


,start_time,bat_landing_to_food,habit,rat_period_start,rat_period_end,seconds_after_rat_arrival,risk,reward,month,sunset_time,hours_after_sunset,season
0,30/12/2017 18:37,16.000000,rat,30/12/2017 18:35,30/12/2017 18:38,108,1,0,0,30/12/2017 16:45,1.870833,0
1,30/12/2017 19:51,0.074016,fast,30/12/2017 19:50,30/12/2017 19:55,17,0,1,0,30/12/2017 16:45,3.100833,0
2,30/12/2017 19:51,4.000000,fast,30/12/2017 19:50,30/12/2017 19:55,41,0,1,0,30/12/2017 16:45,3.107500,0
3,30/12/2017 19:52,10.000000,rat,30/12/2017 19:50,30/12/2017 19:55,111,1,0,0,30/12/2017 16:45,3.126944,0
4,30/12/2017 19:54,15.000000,rat,30/12/2017 19:50,30/12/2017 19:55,194,1,0,0,30/12/2017 16:45,3.150000,0


,time,month,hours_after_sunset,bat_landing_number,food_availability,rat_minutes,rat_arrival_number
0,26/12/2017 16:13,0,-0.5,20,4.000000,0.0,0
1,26/12/2017 16:43,0,0.0,28,4.000000,0.0,0
2,26/12/2017 17:13,0,0.5,25,4.000000,0.0,0
3,26/12/2017 17:43,0,1.0,71,4.000000,0.0,0
4,26/12/2017 18:13,0,1.5,44,3.753857,0.0,0



DTypes 1:
 start_time                    object
bat_landing_to_food          float64
habit                         object
rat_period_start              object
rat_period_end                object
seconds_after_rat_arrival      int64
risk                           int64
reward                         int64
month                          int64
sunset_time                   object
hours_after_sunset           float64
season                         int64
dtype: object

DTypes 2:
 time                   object
month                   int64
hours_after_sunset    float64
bat_landing_number      int64
food_availability     float64
rat_minutes           float64
rat_arrival_number      int64
dtype: object


> Interpretation: Confirm files loaded, shapes make sense, columns match brief.

## 3) Clean Data

In [5]:
# Parse datetimes in dataset1 if present
for col in ["start_time", "rat_period_start", "rat_period_end", "sunset_time"]:
    if col in d1.columns:
        d1[col] = pd.to_datetime(d1[col], errors="coerce", dayfirst=True)

# Parse time in dataset2
if "time" in d2.columns:
    d2["time"] = pd.to_datetime(d2["time"], errors="coerce", dayfirst=True)

# Standardize/derive season in dataset2 if missing
if "season" not in d2.columns or d2["season"].isna().all():
    month_to_season = {
        6: "winter", 7: "winter", 8: "winter",
        9: "spring", 10: "spring", 0: "winter", # Assuming month 0 is December based on d1 head
        1: "winter", 2: "spring", 3: "spring", 4: "spring", 5: "spring" # Add missing months based on d1 head
    }
    if "month" in d2.columns:
        d2["season"] = d2["month"].map(month_to_season)


# Categorical coercion
for col in ["season", "habit"]:
    if col in d1.columns:
        d1[col] = d1[col].astype("category")
    if col in d2.columns:
        d2[col] = d2[col].astype("category")

# NA report
print("NA report dataset1:\n", d1.isna().sum().sort_values(ascending=False).head(20))
print("\nNA report dataset2:\n", d2.isna().sum().sort_values(ascending=False).head(20))

# Simple numeric median imputation to unblock models
def impute_numeric(df):
    for c in df.select_dtypes(include=[np.number]).columns:
        if df[c].isna().any():
            df[c] = df[c].fillna(df[c].median())
    return df

d1 = impute_numeric(d1)
d2 = impute_numeric(d2)
print("\nPost-imputation complete.")

NA report dataset1:
 habit                        41
start_time                    0
bat_landing_to_food           0
rat_period_start              0
rat_period_end                0
seconds_after_rat_arrival     0
risk                          0
reward                        0
month                         0
sunset_time                   0
hours_after_sunset            0
season                        0
dtype: int64

NA report dataset2:
 time                  0
month                 0
hours_after_sunset    0
bat_landing_number    0
food_availability     0
rat_minutes           0
rat_arrival_number    0
season                0
dtype: int64

Post-imputation complete.


> Interpretation: Note any heavy missingness and justify imputation strategy.

## 4) Exploratory Data Analysis

In [6]:

# Risk distribution
if "risk" in d1.columns:
    counts = d1["risk"].value_counts().sort_index()
    plt.figure()
    counts.plot(kind="bar")
    plt.title("Bat risk behaviour (0=avoidance, 1=risk-taking)")
    plt.xlabel("risk")
    plt.ylabel("count")
    save_current_plot("d1_risk_distribution")

# Reward distribution
if "reward" in d1.columns:
    counts = d1["reward"].value_counts().sort_index()
    plt.figure()
    counts.plot(kind="bar")
    plt.title("Bat reward outcome (0=no, 1=yes)")
    plt.xlabel("reward")
    plt.ylabel("count")
    save_current_plot("d1_reward_distribution")

# Risk by season
if {"risk","season"}.issubset(d1.columns):
    print(d1.groupby("season")["risk"].agg(['mean','count']))
    plt.figure()
    d1.groupby("season")["risk"].mean().plot(kind="bar")
    plt.title("Mean risk-taking by season")
    plt.xlabel("season")
    plt.ylabel("mean risk")
    save_current_plot("d1_mean_risk_by_season")

# Rat arrivals by season
if {"season","rat_arrival_number"}.issubset(d2.columns):
    print(d2.groupby("season")["rat_arrival_number"].agg(['mean','median','std','count']))
    plt.figure()
    d2.groupby("season")["rat_arrival_number"].mean().plot(kind="bar")
    plt.title("Mean rat arrivals by season")
    plt.xlabel("season")
    plt.ylabel("mean rat_arrival_number")
    save_current_plot("d2_mean_rat_arrivals_by_season")

# Bat landings vs rat arrivals
if {"rat_arrival_number","bat_landing_number"}.issubset(d2.columns):
    plt.figure()
    plt.scatter(d2["rat_arrival_number"], d2["bat_landing_number"])
    plt.title("Bat landings vs Rat arrivals (30-min windows)")
    plt.xlabel("rat_arrival_number")
    plt.ylabel("bat_landing_number")
    save_current_plot("d2_scatter_bat_vs_rat")


Saved: /content/outputs/d1_risk_distribution.png
Saved: /content/outputs/d1_reward_distribution.png
            mean  count
season                 
0       0.562914    151
1       0.481481    756


/tmp/ipython-input-3304134030.py:23: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(d1.groupby("season")["risk"].agg(['mean','count']))
/tmp/ipython-input-3304134030.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  d1.groupby("season")["risk"].mean().plot(kind="bar")
/tmp/ipython-input-3304134030.py:33: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(d2.groupby("season")["rat_arrival_number"].agg(['mean','me

Saved: /content/outputs/d1_mean_risk_by_season.png
            mean  median       std  count
season                                   
spring  0.535287     0.0  1.118882   1672
winter  0.108647     0.0  0.325495    451
Saved: /content/outputs/d2_mean_rat_arrivals_by_season.png
Saved: /content/outputs/d2_scatter_bat_vs_rat.png


> Interpretation: Describe patterns. Any season with higher rat arrivals or risk-taking?

## 5) Investigation A — Predation Risk Hypothesis

In [7]:

invA_model_summary = None
if "risk" in d1.columns:
    d1["risk"] = d1["risk"].astype(int)

    predictors = []
    if "seconds_after_rat_arrival" in d1.columns:
        predictors.append("seconds_after_rat_arrival")
    if "hours_after_sunset" in d1.columns:
        predictors.append("hours_after_sunset")
    if "habit" in d1.columns: predictors.append("C(habit)")
    if "month" in d1.columns: predictors.append("C(month)")

    if predictors:
        formula = "risk ~ " + " + ".join(predictors)
        try:
            modelA = smf.logit(formula=formula, data=d1.dropna()).fit(disp=False)
            invA_model_summary = modelA.summary2().as_text()
            print(invA_model_summary)

            # Predicted probability vs seconds_after_rat_arrival
            if "seconds_after_rat_arrival" in d1.columns:
                x = np.linspace(d1["seconds_after_rat_arrival"].min(), d1["seconds_after_rat_arrival"].max(), 100)
                base = {}
                if "hours_after_sunset" in d1.columns:
                    base["hours_after_sunset"] = np.full_like(x, d1["hours_after_sunset"].median(), dtype=float)
                if "habit" in d1.columns:
                    base["habit"] = pd.Categorical([d1["habit"].mode().iloc[0]]*len(x), categories=d1["habit"].cat.categories)
                if "month" in d1.columns:
                    base["month"] = pd.Categorical([d1["month"].mode().iloc[0]]*len(x), categories=d1["month"].cat.categories)
                dfp = pd.DataFrame({"seconds_after_rat_arrival": x, **base})
                dfp["pred"] = modelA.predict(dfp)
                plt.figure()
                plt.plot(x, dfp["pred"])
                plt.title("Predicted P(risk-taking) vs seconds_after_rat_arrival")
                plt.xlabel("seconds_after_rat_arrival")
                plt.ylabel("Predicted probability of risk=1")
                save_current_plot("invA_pred_prob_seconds_after_rat_arrival")
        except Exception as e:
            print("Investigation A model failed:", e)

# Chi-square on binned arrival timing
if {"risk","seconds_after_rat_arrival"}.issubset(d1.columns):
    try:
        cutoff = d1["seconds_after_rat_arrival"].median()
        d1["arrival_bin"] = np.where(d1["seconds_after_rat_arrival"] <= cutoff, "early", "late")
        ct = pd.crosstab(d1["arrival_bin"], d1["risk"])
        chi2, p, dof, exp = stats.chi2_contingency(ct)
        print("Contingency table (arrival_bin vs risk):\n", ct)
        print(f"chi2={chi2:.3f}, p={p:.4f}, dof={dof}")
    except Exception as e:
        print("Chi-square failed:", e)


/usr/local/lib/python3.12/dist-packages/statsmodels/discrete/discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


Investigation A model failed: Singular matrix
Contingency table (arrival_bin vs risk):
 risk           0    1
arrival_bin          
early        232  222
late         226  227
chi2=0.089, p=0.7653, dof=1


/usr/local/lib/python3.12/dist-packages/statsmodels/discrete/discrete_model.py:2443: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q * linpred)))


> Interpretation: State whether coefficients suggest more avoidance when rats are proximate. Reference p-values.

## 6) Investigation B — Seasonal Changes

In [8]:

# Risk-taking mean by season + t-tests
if {"season","risk"}.issubset(d1.columns):
    season_risk_share = d1.groupby("season")["risk"].mean()
    print("Mean risk-taking by season:\n", season_risk_share)
    seasons = d1["season"].dropna().unique().tolist()
    if len(seasons) >= 2:
        base = d1.loc[d1["season"]==seasons[0], "risk"].astype(float)
        for s in seasons[1:]:
            other = d1.loc[d1["season"]==s, "risk"].astype(float)
            t,pv = stats.ttest_ind(base, other, equal_var=False)
            print(f"T-test {seasons[0]} vs {s} (risk mean): t={t:.3f}, p={pv:.4f}")

# Seasonal comparisons for dataset2
def ttest_winter_spring(series, name):
    try:
        w = series[d2["season"]=="winter"].dropna().astype(float)
        sp = series[d2["season"]=="spring"].dropna().astype(float)
        if len(w)>1 and len(sp)>1:
            t, p = stats.ttest_ind(w, sp, equal_var=False)
            print(f"T-test {name}: winter vs spring -> t={t:.3f}, p={p:.4f}")
    except Exception as e:
        print(f"T-test {name} failed:", e)

for col in ["rat_arrival_number","rat_minutes","bat_landing_number","food_availability"]:
    if col in d2.columns and "season" in d2.columns:
        print(f"\n{col} by season:\n", d2.groupby("season")[col].agg(["mean","median","std","count"]))
        plt.figure()
        d2.groupby("season")[col].mean().plot(kind="bar")
        plt.title(f"Mean {col} by season")
        plt.xlabel("season")
        plt.ylabel(f"mean {col}")
        save_current_plot(f"d2_mean_{col}_by_season")
        ttest_winter_spring(d2[col], col)


/tmp/ipython-input-1234276474.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  season_risk_share = d1.groupby("season")["risk"].mean()
/tmp/ipython-input-1234276474.py:26: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(f"\n{col} by season:\n", d2.groupby("season")[col].agg(["mean","median","std","count"]))
/tmp/ipython-input-1234276474.py:28: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  d2.groupby("season")[col]

Mean risk-taking by season:
 season
0    0.562914
1    0.481481
Name: risk, dtype: float64
T-test 0 vs 1 (risk mean): t=1.834, p=0.0680

rat_arrival_number by season:
             mean  median       std  count
season                                   
spring  0.535287     0.0  1.118882   1672
winter  0.108647     0.0  0.325495    451
Saved: /content/outputs/d2_mean_rat_arrival_number_by_season.png
T-test rat_arrival_number: winter vs spring -> t=-13.603, p=0.0000

rat_minutes by season:
             mean  median      std  count
season                                  
spring  2.420086     0.0  7.54667   1672
winter  0.416445     0.0  1.72414    451
Saved: /content/outputs/d2_mean_rat_minutes_by_season.png
T-test rat_minutes: winter vs spring -> t=-9.937, p=0.0000

bat_landing_number by season:
              mean  median        std  count
season                                     
spring  32.288278    27.0  26.422805   1672
winter  31.323725    28.0  22.374526    451
Saved: /content/ou

/tmp/ipython-input-1234276474.py:26: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(f"\n{col} by season:\n", d2.groupby("season")[col].agg(["mean","median","std","count"]))
/tmp/ipython-input-1234276474.py:28: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  d2.groupby("season")[col].mean().plot(kind="bar")
/tmp/ipython-input-1234276474.py:26: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(f"\n{col} by season:\n",

Saved: /content/outputs/d2_mean_food_availability_by_season.png
T-test food_availability: winter vs spring -> t=-1.507, p=0.1323


> Interpretation: Summarize significant seasonal differences and relate to winter scarcity vs spring abundance.

## 7) Feature Engineering

In [9]:

if "rat_minutes" in d2.columns:
    d2["rat_pressure"] = d2["rat_minutes"] / 30.0
if {"bat_landing_number","rat_arrival_number"}.issubset(d2.columns):
    d2["landing_rate_per_rat"] = d2["bat_landing_number"] / (d2["rat_arrival_number"] + 1)

eng_cols = [c for c in ["rat_pressure","landing_rate_per_rat"] if c in d2.columns]
if eng_cols:
    print("Engineered feature means by season:\n", d2.groupby("season")[eng_cols].mean())


Engineered feature means by season:
         rat_pressure  landing_rate_per_rat
season                                    
spring      0.080670             27.531437
winter      0.013881             29.545824


/tmp/ipython-input-2564384809.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print("Engineered feature means by season:\n", d2.groupby("season")[eng_cols].mean())


> Interpretation: Explain what these features capture and how they support A/B.

## 8) Optional: Predict Reward (logistic regression)

In [10]:

if "reward" in d1.columns:
    try:
        d1["reward"] = d1["reward"].astype(int)
        preds = []
        if "seconds_after_rat_arrival" in d1.columns: preds.append("seconds_after_rat_arrival")
        if "hours_after_sunset" in d1.columns: preds.append("hours_after_sunset")
        if "habit" in d1.columns: preds.append("C(habit)")
        if "month" in d1.columns: preds.append("C(month)")
        if preds:
            f = "reward ~ " + " + ".join(preds)
            mR = smf.logit(f, data=d1.dropna()).fit(disp=False)
            print(mR.summary2().as_text())
    except Exception as e:
        print("Reward model failed:", e)


Reward model failed: Singular matrix


/usr/local/lib/python3.12/dist-packages/statsmodels/discrete/discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.12/dist-packages/statsmodels/discrete/discrete_model.py:2443: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q * linpred)))


> Interpretation: Note predictors that increase reward probability.

## 9) Save Clean Data and Tables

In [11]:

d1.to_csv(f"{OUTPUT_DIR}/dataset1_clean.csv", index=False)
d2.to_csv(f"{OUTPUT_DIR}/dataset2_clean.csv", index=False)
print(f"Saved clean datasets to {OUTPUT_DIR}")

if "season" in d1.columns and "risk" in d1.columns:
    (d1.groupby("season")["risk"].mean()
       .to_frame("mean_risk")
       .to_csv(f"{OUTPUT_DIR}/table_mean_risk_by_season.csv"))
    print(f"Saved: {OUTPUT_DIR}/table_mean_risk_by_season.csv")

if "season" in d2.columns and "rat_arrival_number" in d2.columns:
    (d2.groupby("season")["rat_arrival_number"].mean()
       .to_frame("mean_rat_arrivals")
       .to_csv(f"{OUTPUT_DIR}/table_mean_rat_arrivals_by_season.csv"))
    print(f"Saved: {OUTPUT_DIR}/table_mean_rat_arrivals_by_season.csv")


Saved clean datasets to /content/outputs
Saved: /content/outputs/table_mean_risk_by_season.csv
Saved: /content/outputs/table_mean_rat_arrivals_by_season.csv


/tmp/ipython-input-2176298291.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  (d1.groupby("season")["risk"].mean()
/tmp/ipython-input-2176298291.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  (d2.groupby("season")["rat_arrival_number"].mean()


In [12]:
import shutil
from google.colab import files

# Create zip archive of the folder
shutil.make_archive("/content/outputs", 'zip', "/content/outputs")

# Download the zip file
files.download("/content/outputs.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

> Interpretation: List which PNGs and CSVs you will insert into the Word report.


## 10) Report Checklist (to include in .docx)
- Introduction, objectives A and B
- Methods: data cleaning, variables, tests, feature engineering
- Results: key tables and figures saved in `/content/outputs`
- Discussion and limitations
- Conclusion and references
- Individual contributions
- Link to code repository + datasets
- Turnitin report + AI usage declaration
